In [ ]:
# credits:
# https://www.kaggle.com/code/bsmit1659/aimo-vllm-accelerated-tot-sc-deepseekmath

In [1]:
!pip uninstall -y torch -q
!pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm -q

# keep data in float16 to avoid OOM
file_path = '/opt/conda/lib/python3.10/site-packages/transformers/models/llama/modeling_llama.py'
with open(file_path, 'r') as file:
    file_contents = file.readlines()
file_contents = [line for line in file_contents if "logits = logits.float()" not in line]
with open(file_path, 'w') as file:
    file.writelines(file_contents)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is incompatible.
dask-cuda 23.8.0 requires pynvml<11.5,>=11.0.0, but you have pynvml 11.5.0 which is incompatible.
raft-dask 23.8.0 requires dask==2023.7.1, but you have dask 2024.3.1 which is incompatible.


In [2]:
from vllm import LLM, SamplingParams
import pandas as pd
from tqdm import tqdm
import gc
import re
import sys
import subprocess
from collections import defaultdict, Counter
import numpy as np
from transformers import (AutoModelForCausalLM,
    AutoTokenizer,
    set_seed)
import torch
import math

llm = LLM(model="/kaggle/input/deepseek-math",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)

tokenizer = llm.get_tokenizer()

good_token = '+'
bad_token = '-'
step_tag = 'ки'

prm_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm')
prm_candidate_tokens = prm_tokenizer.encode(f"{good_token} {bad_token}")[1:] # [648, 387]
step_tag_id = prm_tokenizer.encode(f"{step_tag}")[-1] # 12902
prm_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm',
                                                 torch_dtype=torch.float16,
                                                 device_map="balanced_low_0").eval()

2024-04-27 20:24:36,764	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 04-27 20:24:38 config.py:767] Casting torch.bfloat16 to torch.float16.
INFO 04-27 20:24:38 config.py:381] Using fp8_e5m2 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. But it may cause slight accuracy drop. Currently we only support fp8 without scaling factors and make e5m2 as a default format.
INFO 04-27 20:24:38 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='/kaggle/input/deepseek-math', tokenizer='/kaggle/input/deepseek-math', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=True, kv_cache_dtype=fp8_e5m2, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-27 20:24:40 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-27 20:24:40 selector.py:25] Using XFormers backend.
INFO 04-27 20:26:24 model_runner.py:104] Loading model weights took 12.8725 GB
INFO 04-27 20:26:26 gpu_executor.py:94] # GPU blocks: 177, # CPU blocks: 1092


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [4]:
def eval_prm(candidates):
    # Initialize a list to store all the log probabilities
    all_log_probs = []
    # Process the candidates in batches
    for i in range(len(candidates)):
        # Select a batch of candidates
        input_ids = prm_tokenizer.encode(candidates[i], return_tensors="pt").to("cuda:1")  # Concatenate the padded inputs into a tensor

        with torch.no_grad():
            logits = prm_model(input_ids).logits[:, :, prm_candidate_tokens] # b,l,C
            scores = logits.softmax(dim=-1)[:, :, 0].squeeze() # l
            # Collect the log probabilities from this batch
            all_log_probs.append(scores[-1].item())
    return all_log_probs

In [5]:
stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
stop_words.append("\n")

sampling_params = SamplingParams(temperature=1,
                                 max_tokens=256,
                                 min_tokens=32,
                                 stop=stop_words)

cot_instruction = "\nPlease reason step by step, and put your final answer within \\boxed{}."


n = 5 # beams
samples = 7
max_depth = 24
overlap_threshold = 0.6
all_prompts = []
total_paths = []
total_answers = []

def is_integer(num):
    if isinstance(num, float):
        return num.is_integer()
    elif isinstance(num, int):
        return True
    else:
        return False
    
def is_between_0_and_999(num):
    return 0 <= num <= 999

def prm_prompt(text, current_level):
    return f"Step {str(current_level)}:" + text + ' ки'

def remove_prm_prompt(text):
    pattern = r"Step \d+:"
    text = re.sub(pattern, "", text)
    return text.replace(" ки","")

import re
def extract_number(text):
    patterns = [
        r'The answer is.*\\boxed\{(.*?)\}',
        r"The answer is[:\s]*\$([0-9]+)\$",
        r"The answer is[:\s]*([0-9]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1)
    return 'parse err'

def tot_agg(completed_paths):
    # [(answer,score,len_,current_level),...]
    pass

def get_overlap(nodes_split,node_list):
    max_overlap = float("-inf")
    node_len = len(node_list)
    for previous_split in nodes_split:
        count = 0
        len_ = max(len(previous_split),node_len)
        for i,j in zip(previous_split,node_list):
            count += (i==j)
        count /= len_
        max_overlap = max(max_overlap,count)
    return max_overlap

for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]

    messages = [
        {
            "role": "user",
            "content": problem + cot_instruction
        }
    ]

    base_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    current_level = 1

    current_level_nodes = [base_prompt]
    current_scores = [float('inf')] # for min score
    completed_paths = []
    completed_path_splits = []
    
    while (len(completed_paths) < n) and (current_level < max_depth) and (current_level_nodes):
        # for generation, remove special tokens for PRM
        batch_responses = llm.generate([remove_prm_prompt(t) for t in current_level_nodes]*samples, sampling_params)
        prm_inputs = []
        cumulative_lens = []

        # Collect candidates for reward model evaluation
        for candidate,parent in zip(batch_responses,current_level_nodes*samples):
            prm_input = parent + prm_prompt(candidate.outputs[0].text,current_level)
            cumulative_tokens = len(candidate.prompt_token_ids) + len(candidate.outputs[0].token_ids)
            prm_inputs.append(prm_input)
            cumulative_lens.append(cumulative_tokens)
        # Get the indices of unique elements in prm_inputs
        unique_indices = [i for i, x in enumerate(prm_inputs) if prm_inputs.index(x) == i]
        prm_inputs = [prm_inputs[i] for i in unique_indices]
        current_scores = [(current_scores*samples)[i] for i in unique_indices]
        cumulative_lens = [cumulative_lens[i] for i in unique_indices]
        
        # Batch reward model evaluation
        prm_scores = eval_prm(prm_inputs)
        prm_scores = [min(old,new) for old,new in zip(current_scores,prm_scores)]
        next_level_nodes = []
        next_scores = []
        nodes_split = []
        
        # Prune to keep only the top 'n' candidates based on scores
        combined = list(zip(prm_inputs,prm_scores,cumulative_lens))
        combined.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their scores
        for node,score,len_ in combined:
            answer = extract_number(remove_prm_prompt(node))
            if answer == 'parse err': # not finished
                if len_ > 2048: continue # max out len_
                node_list = node.split(" ки")
                print(get_overlap(nodes_split,node_list))
                if get_overlap(nodes_split,node_list) < overlap_threshold:
                    next_level_nodes.append(node)
                    next_scores.append(score)
                    nodes_split.append(node_list)
                else: continue
                if len(next_level_nodes) == n: break
            else: # finished
                node_list = node.split(" ки")
                if get_overlap(completed_path_splits,node_list) < overlap_threshold:
                    try:
                        answer = eval(answer)
                        if is_integer(answer) and is_between_0_and_999(answer):# correct format
                            completed_paths.append((answer,score,len_,current_level))
                            completed_path_splits.append(node_list)
                    except: # bad eval
                        continue
        # if current_level_nodes is empty, all max out or err out. exit loop
        current_scores, current_level_nodes = next_scores, next_level_nodes
        current_level += 1
    
    print(f'problem {i}, sol {completed_paths}')
    total_paths.append(completed_paths)
#     total_answers.append(final_answer)

Processed prompts: 100%|██████████| 7/7 [00:06<00:00,  1.05it/s]


-inf
0.5
0.5
0.5
0.5



Processed prompts: 100%|██████████| 35/35 [00:24<00:00,  1.45it/s]


-inf
0.3333333333333333
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.3333333333333333
0.6666666666666666
0.6666666666666666
0.3333333333333333
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.6666666666666666
0.3333333333333333



Processed prompts: 100%|██████████| 35/35 [00:24<00:00,  1.42it/s]


...

-inf
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.058823529411764705
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.058823529411764705
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.058823529411764705
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353
0.9411764705882353



Processed prompts: 100%|██████████| 28/28 [01:04<00:00,  2.31s/it]


-inf
0.05555555555555555
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.05555555555555555
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444
0.9444444444444444



100%|██████████| 10/10 [1:57:23<00:00, 704.32s/it]

-inf
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.05263157894736842
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.05263157894736842
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
0.9473684210526315
problem 9, sol [(99, 0.56982421875, 285, 4), (99, 0.51953125, 435, 8), (99, 0.48828125, 690, 12), (72, 0.352294921875, 1917, 16), (100, 0.414794921875, 1061, 18), (127, 0.3701171875, 973, 18)]


In [7]:
total_paths

[[(28, 0.71240234375, 724, 8),
  (52, 0.658203125, 750, 11),
  (41, 0.6259765625, 822, 11),
  (28, 0.418701171875, 866, 11),
  (36, 0.385009765625, 1301, 17)],
 [(480, 0.611328125, 497, 4),
  (9, 0.59619140625, 447, 4),
  (6, 0.67919921875, 415, 5),
  (898, 0.20947265625, 687, 5),
  (6, 0.67919921875, 451, 6)],
 [(17, 0.615234375, 659, 8),
  (9, 0.515625, 816, 8),
  (5, 0.64404296875, 655, 9),
  (2, 0.56982421875, 583, 9),
  (146, 0.48046875, 746, 9)],
 [(800, 0.16455078125, 1262, 18)],
 [(310, 0.82421875, 294, 1),
  (310, 0.8154296875, 286, 3),
  (310, 0.8154296875, 293, 4),
  (310, 0.763671875, 473, 5),
  (310, 0.763671875, 617, 6),
  (310, 0.57763671875, 363, 6)],
 [(199, 0.56591796875, 386, 6),
  (199, 0.48046875, 372, 8),
  (101, 0.51171875, 490, 9),
  (197, 0.449462890625, 753, 9),
  (199, 0.42626953125, 453, 9)],
 [(194, 0.6689453125, 326, 3),
  (97, 0.49609375, 386, 3),
  (97, 0.84375, 459, 5)],
 [(100, 0.7216796875, 598, 7),
  (256, 0.69921875, 595, 8),
  (256, 0.6298828125, 6

In [ ]:
total_paths

In [7]:
len(set(current_level_nodes)),len(current_level_nodes),len(set(prm_inputs)),len(prm_inputs)

(5, 5, 32, 32)

In [ ]:
df['answer'] = final_answers

In [ ]:
df

In [ ]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [ ]:
df[['id','answer']].head()